## <u>2. Datenaufbereitung Bureau

In diesem Dokument werden für die Untersuchung unwichtige Variablen aus dem Bureau-Datensatz gelöscht. Dabei werden zuerst die kategorischen Variablen (nominale & ordinale) betrachtet, um anschließend die metrischen Variablen zu betrachten. Im Gegensatz zu den Application-Daten liegt in dem Bureau-Datensatz eine 1:N-Beziehung vor, da ein Kreditnehmer in der Vergangenheit mehrere Kredite gehabt haben kann. Dies erfordert die Gruppierung der historischen Daten.

*Vorgehensweise kategorische Variablen:*
- Gruppierung der Variablen
- Löschung von Variablen mit mehr als 60% fehlenden Daten
- Löschung von nominalen Variablen mit weniger als 5pP relativer Anteilsdifferenz zwischen Paybacks und Defaults
- Bildung von Korrelationsclustern (Kontingenzkoeffizent bei nominalen Daten)
- Löschung von Variablen ohne kausalen Einfluss auf die Kreditwürdigkeitsbestimmung

*Vorgehensweise metrischer Variablen:*
- Gruppierung der Variablen
- Löschung von Variablen mit mehr als 60% fehlenden Daten
- Bildung von Korrelationsclustern (Pearson-Korrelationskoeffizient)
- Löschung von Variablen ohne kausalen Einfluss auf die Kreditwürdigkeitsbestimmung

## Initialisierung

In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from sklearn.linear_model import LogisticRegression

from IPython.display import display, Markdown

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
bureau = pd.read_csv(DATASET_DIR / "bureau.csv")
description = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
des = description.loc[description['Table']=="bureau.csv", "Row":"Special"]

In [5]:
bureau = pd.merge(bureau, app_train[["SK_ID_CURR","TARGET"]] ,on="SK_ID_CURR")

In [6]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,TARGET
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,0


In [7]:
# Spalten die innerhalb der Aufbereitung nicht verändert werden können
skip = ["TARGET", "SK_ID_CURR", "SK_ID_BUREAU"]

In [8]:
# nominale und metrische Spalten
n_heads = [element for element in bureau.columns if bureau[element].dtype.name == "object"]
m_heads = [element for element in bureau.columns if bureau[element].dtype.name != "object"]

## <u>kategorische Variablen

In [9]:
df = bureau[["SK_ID_BUREAU", "SK_ID_CURR", "TARGET"] + n_heads].copy()

In [10]:
des[des["Row"] == "FLAG_EMP_PHONE"]

,Row,Description,Special


In [11]:
df.head()

,SK_ID_BUREAU,SK_ID_CURR,TARGET,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE
0,5714462,215354,0,Closed,currency 1,Consumer credit
1,5714463,215354,0,Active,currency 1,Credit card
2,5714464,215354,0,Active,currency 1,Consumer credit
3,5714465,215354,0,Active,currency 1,Credit card
4,5714466,215354,0,Active,currency 1,Consumer credit


# Informationsgehalt:
- Anzahl Kredite je Kreditnehmer
- Kreditstatus der Kredite
- Kredittyp

In [12]:
# Anzahl Kredite

cnt = df[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby(by=["SK_ID_CURR"]).count()
cnt.columns = ["CNT_BURAEU"]
cnt.head()

,CNT_BURAEU
SK_ID_CURR,
100002,8
100003,4
100004,2
100007,1
100008,3


In [16]:
len(cnt)

263491

In [15]:
cnt.mean()

CNT_BURAEU    5.561196
dtype: float64

Kreditnehmer 100002 hat in seiner Kredithistorie 8 Kredite bei externen Kreditgebern in Anspruch genommen

In [ ]:
# Kreditstatus

status = df[["SK_ID_CURR", "CREDIT_ACTIVE"]].groupby(by=["SK_ID_CURR", "CREDIT_ACTIVE"]).size().unstack(fill_value=0)
status.head()

Zum Zeitpunkt der Kreditvergabe im Application-Datensatz hat Kreditnehmer 100002 2 aktive Kredite bei externen Kreditnehmern.

In [ ]:
# Kredittyp

typ = df[["SK_ID_CURR", "CREDIT_TYPE"]].groupby(by=["SK_ID_CURR", "CREDIT_TYPE"]).size().unstack(fill_value=0)
typ.head()

Kreditnehmer 100002 hat in seiner 4 Konsumentenkredite und 4 Kreditkarten in Anspruch genommen.

In [ ]:
result = pd.DataFrame(index=bureau.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

In [ ]:
result = pd.merge(result, cnt, how="left", left_index=True, right_index=True)
result = pd.merge(result, status, how="left", left_index=True, right_index=True)
result = pd.merge(result, typ, how="left", left_index=True, right_index=True)

In [ ]:
df = result
df.head()

In [ ]:
target = app_train[["SK_ID_CURR", "TARGET"]]
target = target.set_index("SK_ID_CURR")

In [ ]:
df = pd.merge(df, target, left_index=True, right_index=True)

### Löschung der Spalten mit weniger als 40% ausgefüllten Daten

In [ ]:
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
result

Es gibt keine kategorischen Variablen mit mehr als 60% fehlenden Daten

In [ ]:
df = df.drop(result.header.values, axis=1)

### Unterscheidbarkeit von mindestens 5pP einer Kategorie

In [ ]:
ID_Payback = df[df["TARGET"] == 0].index.values
ID_Default = df[df["TARGET"] == 1].index.values

In [ ]:
payback = df.loc[ID_Payback]
default = df.loc[ID_Default]

In [ ]:
result = {
    "head" : [],
    "cat" : [],
    "payback" : [],
    "default" : [],
    "diff" : []
}

for head in df.columns.values:
    df1 = payback[head].value_counts().rename_axis(head).reset_index(name='payback')
    df2 = default[head].value_counts().rename_axis(head).reset_index(name='default')
    
    df1["payback"] = df1["payback"]/df1["payback"].sum()*100
    df2["default"] = df2["default"]/df2["default"].sum()*100
    
    df_ = df1.merge(df2, how="outer", on=head)
    
    df_["diff"] = (df_["default"]-df_["payback"])
    
    df_ = df_.sort_values("diff", ascending=False)
    
    for diff in df_["diff"]:
        if np.isnan(diff):
            continue
        if diff > 5 or diff < -5:
            row = df_.loc[df_["diff"] == diff]
            cat = row[head][row[head].index[0]]
            
            result["head"].append(head)
            result["cat"].append(cat)
            result["payback"].append(round(row["payback"].values[0],2))
            result["default"].append(round(row["default"].values[0],2))
            result["diff"].append(round(diff,2))

result = pd.DataFrame(result)
result.sort_values("diff", ascending=False)

Aus den nominalen Variablen kristallisieren sich zwei Unterscheidungsmerkmale heraus. Defaults sind häufiger Kreditnehmer ohne Kredithistorie bei externen Kreditgebern. Darüber hinaus haben sie paralell zum Kreditbeginn häufiger einen oder mehr externe Kredite, die sie paralell abbezahlen.
Paybacks hingegen haben häufiger keinen aktiven Kredit bei externen Kreditgebern den sie paralell abbezahlen. Zusätzlich haben sie bei externen Kreditgebern eine Kredithistorie.

In [ ]:
remove = [head for head in df.columns.values if head not in list(result["head"].unique()) + skip]

In [ ]:
remove

In [ ]:
df = df.drop(remove, axis=1)

In [ ]:
df = df.drop(["TARGET"], axis=1)

In [ ]:
df = df.add_prefix("B_")

In [ ]:
df.head()

In [ ]:
des[des["Row"] == "CREDIT_ACTIVE"]

### Speichern der kategorischen Werte

In [ ]:
cats = df

## <u>metrische Variablen

In [ ]:
df = bureau[m_heads].copy()

In [ ]:
df[df["SK_ID_CURR"] == 298038].head()

In [ ]:
df["DAYS_CREDIT_ENDDATE"].isna().sum()

# Informationsgehalt:
(Es werden nur Kredite betrachtet, die maximal ein halbes Jahr in der Vergangenheit liegen.)
- Summen: CREDIT_DAY_OVERDUE (überzogene Tage), 
- Durchschnitt: DAYS_CREDIT_ENDDATE (verbleibende Laufzeit), AMT_CREDIT_SUM (Kredithöhe), AMT_CREDIT_SUM_DEBT (Schuldenhöhe), AMT_CREDIT_SUM_OVERDUE (überzogener Betrag), AMT_ANNUITY (Zahlungsbeitrag pro Jahr), DEBT_PER_LIMIT (Verschuldungsquote)

In [ ]:
df = df[df["DAYS_CREDIT_ENDDATE"] > -180]

In [ ]:
result = pd.DataFrame(index=bureau.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

In [ ]:
# Durchschnitte
mean_heads = ["DAYS_CREDIT_ENDDATE"]

for head in mean_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.groupby(by=["SK_ID_CURR"]).mean()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [ ]:
# Summe
sum_heads = ["CREDIT_DAY_OVERDUE", "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_OVERDUE", "AMT_ANNUITY"]

for head in sum_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [ ]:
df = result
df.head()

### Löschung der Spalten mit weniger als 40% ausgefüllten Daten

In [ ]:
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
result

In [ ]:
df = df.drop(result.header.values, axis=1)

Es gibt keine Variablen mit mehr als 60% fehlenden Daten.

### Bildung von Korrelationsclustern

In [ ]:
c = df.corr(method='pearson') * 100

In [ ]:
c

In [ ]:
families = []
for i, row in c.iterrows():
    r = row[row > 80]
    if len(r) > 1 and set(r.index) not in families:
        families.append(set(r.index))

for A in families:
    for B in families:
        if A == B:
            continue
        if A.issubset(B):
            families.remove(A)
families

In [ ]:
result = {
          "family":[],
          "head":[],
          "r2":[],
          "na":[],
          "rate":[]
         }

for i, family in enumerate(families):
    headers = list(family)
    
    result["family"].append("")
    result["head"].append("")
    result["r2"].append("")
    result["na"].append("")
    result["rate"].append("")
    
    for head in headers:
        d = df[["TARGET"] + [head]]
        na = d[head].isna().sum() / len(d) * 100
        d = d.dropna()
        x = d[[head]]
        y = d[["TARGET"]]
        model = LogisticRegression().fit(x, y.values.ravel())
        r2 = round(model.score(x,y),5)
        
        result["family"].append(i)
        result["head"].append(head)
        result["r2"].append(round(r2,5))
        result["na"].append(round(na,2))
        result["rate"].append(r2/na)
    
result = pd.DataFrame(result)
result       

In [ ]:
df.head()

Es bilden sich keine Korrelationscluster

### Betrachtung der Kausalität

In [ ]:
result = {
    "head":[],
    "des":[]
}

for head in df.columns.values:
    if head in skip:
        continue
    result["head"].append(head)
    result["des"].append(des[des["Row"] == head]["Description"])
    
result = pd.DataFrame(result)
result

Es kann nicht angenommen werden, dass eine Variable nicht auf die Kreditfähigkeit wirkt.

### Ergebnis

In [ ]:
df = df.add_prefix("B_")

### Speichern der metrischen Werte

Zusammenführung der Kreditanzahl und der nominalen Variablen

In [ ]:
cats = pd.merge(cnt, cats, left_index=True, right_index=True)

Zusammenführen der metrischen und kategorischen Variablen

In [ ]:
df = pd.merge(cats, df, left_index=True, right_index=True)

In [ ]:
df.head()

In [ ]:
df.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "bureau.csv")